In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""POJ_INZ_YOLOTINY

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/13GzEyjp-VvmVb_QiObtmDq9-sAnK5IMI
"""

#from google.colab import drive
#drive.mount('/content/drive')
#
## Commented out IPython magic to ensure Python compatibility.
!pip install tensorflow==1.4.0
!pip install keras==1.2.2
!sudo apt install tesseract-ocr
!pip install pytesseract
# %matplotlib inline
!pip install opencv-python==3.4.2.16
#!

#import sys
#sys.path.remove('/home/usher/rosintr_ws/devel/lib/python2.7/dist-packages')
#sys.path.remove('/home/usher/catkin_ws/devel/lib/python2.7/dist-packages')
#sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')




import keras
keras.__version__
from keras.models import load_model
import math
import matplotlib as mpl
#mpl.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import sys
#sys.path.insert(0, '/content/drive/My Drive/projekt_inż')
import pytesseract
import time
import cv2
from moviepy.editor import VideoFileClip
from keras.models import load_model
from scipy.special import expit as sigmoid
import shutil
import os
import random
from collections import Counter
from PIL import Image
from tqdm import tqdm_notebook as tqdm

model = load_model('/content/drive/My Drive/projekt_inż/YOLO_v3_weigths/tiny-yolo-voc.h5')

class BoundBox:
  
  def __init__(self, classes):
    self.x, self.y = float(), float()
    self.w, self.h = float(), float()
    self.c = float()
    self.class_num = classes
    self.probs = np.zeros((classes,))

def overlap(x1,w1,x2,w2):
  l1 = x1 - w1 / 2.
  l2 = x2 - w2 / 2.
  left = max(l1, l2)
  r1 = x1 + w1 / 2.
  r2 = x2 + w2 / 2.
  right = min(r1, r2)
  return right - left

def box_intersection(a, b):
  w = overlap(a.x, a.w, b.x, b.w)
  h = overlap(a.y, a.h, b.y, b.h)
  if w < 0 or h < 0:
    return 0
  area = w * h
  return area

def box_union(a, b):
  i = box_intersection(a, b)
  u = a.w * a.h + b.w * b.h - i
  return u

def box_iou(a, b):
  return box_intersection(a, b) / box_union(a, b)

def prob_compare(box):
  return box.probs[box.class_num]

def prob_compare2(boxa, boxb):
  if (boxa.pi < boxb.pi):
    return 1
  elif(boxa.pi == boxb.pi):
    return 0
  else:
    return -1
def expit(x):
  return 1. / (1. + np.exp(-x))

def _softmax(x):
  e_x = np.exp(x - np.max(x))
  out = e_x / e_x.sum()
  return out

def postprocess2(im, save = True):
  image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  resized_image = cv2.resize(im,(416,416))
  image_data = np.array(resized_image, dtype='float32')
  image_data /= 255.
  image_data = np.expand_dims(image_data, 0)
  net_out = (model.predict(image_data))[0]
  #print(net_out.shape)
  " takes net ouput, draw net_out, save img"
  H, W, _ = [13,13,125]
  threshold = 0.3
  C, B = 20, 5
  anchors = [1.08, 1.19, 3.42, 4.41, 6.63, 11.38, 9.42, 5.11, 16.62, 10.52]
  net_out = net_out.reshape([H, W, B, -1])
  rois = list()
  boxes = list()
  for row in range(H):
    for col in range(W):
      for b in range(B):
        bx = BoundBox(C)
        bx.x, bx.y, bx.w, bx.h, bx.c = net_out[row, col, b, :5]
        bx.c = expit(bx.c)
        bx.x = (col + expit(bx.x)) / W
        bx.y = (row + expit(bx.y)) / H
        bx.w = math.exp(bx.w) * anchors[2 * b + 0] / W
        bx.h = math.exp(bx.h) * anchors[2 * b + 1] / H
        classes = net_out[row, col, b, 5:]
        bx.probs = _softmax(classes) * bx.c
        bx.probs *= bx.probs > threshold
        boxes.append(bx)
  # non max suppress boxes
  for c in 5,6,13:
    for i in range(len(boxes)):
      boxes[i].class_num = c
    boxes = sorted(boxes, key=prob_compare)
    for i in range(len(boxes)):
      boxi = boxes[i]
      if boxi.probs[c] == 0:
        continue
      for j in range(i + 1, len(boxes)):
        boxj = boxes[i]
        if box_iou(boxi, boxj) >= .4:
          boxes[j].probs[c] = 0.  
  labels = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
            'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 
            'dog', 'horse', 'motorbike', 'person', 'pottedplant',
            'sheep', 'sofa', 'train', 'tvmonitor']
  if type(im) is not np.ndarray:
    imgcv = cv2.imread(im)
  else:
    imgcv = im
    h, w, _ = imgcv.shape

  for b in boxes:
    max_indx = np.argmax(b.probs)
    max_prob = b.probs[max_indx]
    label = 'object' * int(C < 2)
    label += labels[max_indx] * int(C>1)
    if max_prob > threshold:
      left  = int ((b.x - b.w/2.) * w)
      right = int ((b.x + b.w/2.) * w)
      top   = int ((b.y - b.h/2.) * h)
      bot   = int ((b.y + b.h/2.) * h)
      if left < 0:
        left = 0
      if right > w - 1:
        right = w - 1
      if top < 0:
        top = 0
      if bot > h - 1:
        bot = h - 1
      car_roi = imgcv[top:bot, left:right, :]
      rois.append(car_roi)
      thick = 2
      cv2.rectangle(imgcv, (left, top), (right, bot), (0,255,0), thick)
      mess = '{}'.format(label)

  return imgcv, rois


class VideoApp():

  def __init__(self, video, savfilename = 'video_pred', h_boundary = 200, w_boundary = 340):
      
    self.h_boundary = h_boundary
    self.w_boundary = w_boundary
    self.savfilename = savfilename
    self.video = video
    self.pred_dict = {}
    self.roi_frames_list = list()
    self.roi_plate_list = list()
    self.num = 1
    self.length = None
    self.frames = None
    self.size = None
    self.new_frame = None
    self.pred_dict_state = False
    
  def plateExtrVid(self, roi_pred):

    '''
    Przyjmuje listę z wyciętymi częściami img, w których wykryto obiekt,
    poszukuje ROI, które w jak największym stopniu przypomina tablice rej.
    i zwraca listę z przypuszcalnymi tabl rej. wyciągnietmi z każdej ramki
    return: [list]
    '''
    pred = np.copy(roi_pred)
    img_test = pred
    img_test = cv2.resize(img_test, (378,325))
    img_test = img_test[100:250,100:300]
    gray = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.GaussianBlur(gray, (3, 3), 0)
    th, dst = cv2.threshold(img_gray, 90, 200, cv2.THRESH_BINARY)
    _, ctrs, hier_ = cv2.findContours(dst.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(cnt) for cnt in ctrs]
    for rect in rects:
      roi = img_test[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
      h ,w ,color = roi.shape
      if h < w:
        if 30 < h < 50:
          if 60 < w < 160:
          
            return roi
      else:
        continue
  
  def plateRecoVid(self, img):

    '''
    przyjmuje listę, z wyciętymi tablicami rej. przekształca do black/white
    i ropoznaje znaki, zapisuje w liście, poczym wbiera 
    najczęściej występującą predykcję
    return: 'string'
    '''
    #print('Potential plates in prediction: '+str(len(img)))
    #wartości do sprawdzenia thesholdów
    min_val = range(90,220,10)
    max_val = range(190,330,10)
    #list ze wszystkimi detekcjami
    pred_outcome = []
    h, w, _ = img.shape
    img = img[0:h,(w//15):w,:]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.medianBlur(gray,1)
    #sprawdzenie różnych thresholdów
    for low in min_val:
      for high in max_val:
        th, dst = cv2.threshold(gray, low, high, cv2.THRESH_BINARY) #100 #180    
        text = pytesseract.image_to_string(dst, lang='eng',
        config='--psm 1 --oem 3 -c tessedit_char_whitelist=0123456789BCDEFGHIJKLMNOPQRSTUVWXYZ')
        if 6 < len(text) < 10:
          if text[2] == ' ':
          
            pred_outcome.append(text)
        if len(pred_outcome) > 8 or (high==320 and low ==210):
          if pred_outcome is None:
            return []
          else:
            return pred_outcome
        else:
          continue
  
  def videoFrame(self):
    # utworzenie obiektu video
    cap = cv2.VideoCapture(self.video)
    # list ze wszystkimi klatkami video
    frames = []
    # sprawdza czy video załadowane
    if cap.isOpened():
      print('Video in progress...')
      # ilość klatek w video
      length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      print('Number of frames: '+str(length))
    else:
      print("ERROR")
    # kiedy video otwarte
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is not None:
          h, w, l = frame.shape
          size = (w, h)
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
        if not ret:
            break;        
        frames.append(frame)
    self.frames = frames
    self.length = length
    self.size = size
  
  def featureComp(self, img1 ,img2):
    try:
      MIN_MATCH_COUNT = 5
      # Initiate SIFT detector
      sift = cv2.xfeatures2d.SIFT_create()
      # find the keypoints and descriptors with SIFT
      kp1, des1 = sift.detectAndCompute(img1,None)
      kp2, des2 = sift.detectAndCompute(img2,None)
      FLANN_INDEX_KDTREE = 0
      index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
      search_params = dict(checks = 50)
      flann = cv2.FlannBasedMatcher(index_params, search_params)
      matches = flann.knnMatch(des1,des2,k=2)
      # store all the good matches as per Lowe's ratio test.
      good = []
      for m,n in matches:
          if m.distance < 0.7*n.distance:
              good.append(m)
      if len(good)>MIN_MATCH_COUNT:
        return True
      else:
        return False
    except:
      return False
  def cnnFrames(self):
    self.out = cv2.VideoWriter(self.savfilename+'.mp4' ,cv2.VideoWriter_fourcc(*'mp4v'), 25, self.size)
    self.progress_bar_cnn = tqdm(total = self.length)
    for frame in self.frames:
          
        self.progress_bar_cnn.update(1)
        self.new_frame, roi_frame = postprocess2(frame)
        self.new_frame = cv2.cvtColor(self.new_frame, cv2.COLOR_BGR2RGB)
        self.roi_frames_list.append(roi_frame)
        self.out.write(self.new_frame)
    self.out.release()
    self.progress_bar_cnn.close()
  def extrPlate(self):
      
    self.progress_bar_rois = tqdm(total = len(self.roi_frames_list))
    for roi in self.roi_frames_list:
        self.progress_bar_rois.update(1)
        for box in roi:
            h,w,l = box.shape
            if h > self.h_boundary and w > self.w_boundary:
                roi_plate = self.plateExtrVid(box)
                if roi_plate is not None:
                    self.roi_plate_list.append(roi_plate)
                else:
                    continue
            else:
                continue
    self.progress_bar_rois.close()
  
  def predPlate(self):
      
    self.progress_bar_pred = tqdm(total = len(self.roi_plate_list))
     
    #while self.cur_frame < len(self.roi_plate_list):
    for roi_plate in self.roi_plate_list:
      
      if bool(self.pred_dict) == True:
        # jeśli tablica rej z poprzedniej klatki jest taka sama jak kolejna
        if self.featureComp(self.pred_dict[self.num][0],roi_plate):
          # ogranieczenie, żeby nie zapisywał więcej niż 3 ropoznania z tablic 
          if len(self.pred_dict[self.num][1:]) <= 2:
            rec_text = self.plateRecoVid(roi_plate)
            self.pred_dict[self.num].append(rec_text)
            #self.cur_frame += 1
            self.progress_bar_pred.update(1)
          # gdy pojawia się ta sama rej więcej niż 3 razy
          else:
            #self.cur_frame += 1
            self.progress_bar_pred.update(1)
            continue
        else:
          rec_text = self.plateRecoVid(roi_plate)
          rec_text_test = Counter(rec_text)
          if len(rec_text) > 4:
            if rec_text_test[max(rec_text_test)] > 6:
              self.num+=1
              self.pred_dict[self.num] = [roi_plate, rec_text]
              #self.cur_frame += 1
              #plt.imshow(roi_plate)
              #plt.show()
              self.progress_bar_pred.update(1)
            else:
              #self.cur_frame += 1
              self.progress_bar_pred.update(1)
              continue
          else:
            #self.cur_frame += 1
            self.progress_bar_pred.update(1)
            continue
        #kiedy dict() nie zawiera żadnych danych
      if bool(self.pred_dict) == False:
        rec_text = self.plateRecoVid(roi_plate)
        rec_text_test = Counter(rec_text)
        if len(rec_text) > 4:
          if rec_text_test[max(rec_text_test)] > 6:
            self.pred_dict[self.num] = [roi_plate, rec_text]
            #plt.imshow(roi_plate)
            #plt.show()
            #self.cur_frame += 1
            self.progress_bar_pred.update(1)
          else:
            #self.cur_frame += 1
            self.progress_bar_pred.update(1)
            continue
        else:
          #self.cur_frame += 1
          self.progress_bar_pred.update(1)
          continue
      else:
        #self.cur_frame += 1
        self.progress_bar_pred.update(1)
        continue

  def getDict(self):
    self.progress_bar_pred.close()
    return self.pred_dict

  def plate_verify(self,predicted_plates):
  
    '''
    Pobiera listę z str() znaków rozpoznanych przez 
    model i wyznacza właściwą poprzez porównanie wszystkich
    dostępnych rozwiązań
    '''
    #predicted_plates = filter(None,predicted_plates)
    #predicted_plates = list(predicted_plates)
    exc_set = []
    rec_word = []

    for num in range(len(predicted_plates)):
      for plate in predicted_plates[num]:
        if ' ' in plate:
          rec_word.append(plate)
        else:
          continue
    set_words = list(set(rec_word))
    res = "" 
  
    for i in set_words[0]: 
        if i in set_words[1]:
            res += i 
    return res
    

  def flowchartApp(self):
    print('Flowchart Started...')
    self.videoFrame()
    print('Preparing frames')
    self.cnnFrames()
    print('Preparing boxes')
    self.extrPlate()
    print('Predicting plates')
    self.predPlate()
    print('Finished')
    roi_plate_dict = self.getDict()
    print (roi_plate_dict)
    pred_chars = roi_plate_dict[1][1:]
    print (pred_chars)
    plt.imshow(roi_plate_dict[1][0]) 
    plt.show()
    # for key in roi_plate_dict.keys():
    #   pred_chars = roi_plate_dict[key][1:]
    #   # word = self.plate_verify(pred_chars)
    #   # print(word)
    #   # print (set(pred_chars))
    #   plt.imshow(roi_plate_dict[key][0]) 
    #   plt.show()

  #frames, size, length = video2Frame(video2)
#
  #app = VideoApp()
  #app.videoFrame()
  #print('preparing frames')
  #app.cnnFrames()
  #print('prep boxes')
  #app.extrPlate()
  #print('predicting plates')
  #app.predPlate()
  #print('finished')
  #roi_plate_dict = app.getDict()
  #print (roi_plate_dict)
  ##roi_plate_dict = video_reco_plate(frames, size, length)#, h_boundary = 400, w_boundary = 540)
  #for key in roi_plate_dict.keys():
  #  pred_chars = roi_plate_dict[key][1:]
  #  #word = plate_verify(pred_chars)
  #  #print(word)
  #  print (pred_chars)
  #  plt.imshow(roi_plate_dict[key][0]) 
  #  plt.show()

  #pred_chars = roi_plate_dict[1][1:]
  #word = plate_verify(pred_chars)
  #print(word)
  #plt.imshow(roi_plate_dict[1][0])
  #plt.show() 

In [ ]:
if __name__ == "__main__":
  video = '/content/drive/My Drive/projekt_inż/TEST_VIDEO_4RECO.mp4'

  app = VideoApp(video)
  app.flowchartApp()